# DEEP HAPPINESS DETECTOR PT. 2
## building an ensemble 
go check pt 1 here: https://www.kaggle.com/gianmarcoguarnier/deep-happiness-detector-pt-1

After having built a single binary classificator in part 1 we can move to part 2: <br>
In this section we are going to train an **ensemble classifcator** where each voter is a binary classificator. <br>
We can use a global variable to train 5 binary classificators 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Importing Deep Learning Libraries
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D,Input
from keras.models import Model,Sequential 
from tensorflow.keras.optimizers import Adam,SGD,RMSprop

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

## TF Data pipeline

In [2]:
train_path = '../input/face-expression-recognition-dataset/images/train'
valid_path = '../input/face-expression-recognition-dataset/images/validation'

Original images are sized 48x48 pixels. The resolution is pretty bad. We can up-sample to 256x256 by uploading the data with<br>
tf.keras.utils.image_from_directory(directory)

In [3]:
TRAIN = tf.keras.utils.image_dataset_from_directory(train_path)    
VALID = tf.keras.utils.image_dataset_from_directory(valid_path)    
print(TRAIN)

TRAIN_unbatched = TRAIN.unbatch()
VALID_unbatched = VALID.unbatch()

import gc
del TRAIN,VALID
gc.collect()

Found 28821 files belonging to 7 classes.


2022-09-19 22:18:50.282376: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 22:18:50.379601: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 22:18:50.380394: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 22:18:50.389861: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Found 7066 files belonging to 7 classes.
<BatchDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int32)>


0

## function the needs to be called

In [4]:
def rename_labl(x,y):
    label = np.int32(-1)
    if y==keep[0]: label = np.int32(1)
    if y==keep[1]: label = np.int32(0)    
    return x,label

def drop_extra_classes(x,y):
        if y == keep[0]: return True
        if y == keep[1]: return True
        return False

## Training and saving 5 models 

In [ ]:
labels = {0:'angry', 1:'disgust', 2:'fear', 3:'happy', 4:'neutral', 5:'sad',6:'surprise'}
fig, ax = plt.subplots(ncols= 3, nrows=3)
for i in range(6):
    
    keep = [3,2]
    
    train = TRAIN_unbatched.filter(drop_extra_classes)
    valid = VALID_unbatched.filter(drop_extra_classes)
    
    # normalizing...
    train = train.map(lambda x,y: (x/255,y))
    valid = valid.map(lambda x,y: (x/255,y))
    
    # rename label to 1-0
    def rename_labl(x,y):
        label = np.int32(-1)
        if y==keep[0]: label = np.int32(1)
        if y==keep[1]: label = np.int32(0)    
        return x,label
    
    # batch back up
    train = train.batch(batch_size = 50)
    valid = valid.batch(batch_size = 50)
    
    # model init
    model = Sequential()

    #1st CNN layer
    model.add(Conv2D(64, (3,3), 1, activation='relu', input_shape=(256,256,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    #2nd CNN layer
    model.add(Conv2D(128,(3,3),padding = 'same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout (0.25))

    model.add(Flatten())

    #Fully connected 1st layer
    model.add(Dense(64))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    #Fully connected 2st layer
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(1, activation='sigmoid'))
    model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])
    
    
    # Regularization
    logdir='logs'
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
    reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=3,verbose=0,min_delta=0.0001)
    early_stopping = EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)
    
    hist = model.fit(train, verbose=0, epochs=50, validation_data=valid, callbacks=[tensorboard_callback,reduce_learningrate, early_stopping])
    
    # visualize training overview
    ax[i].title(labels[i])
    ax[i].ylabel('Accuracy', fontsize=16)
    ax[i].xlabel('Epochs',fontsize=16)
    ax[i].plot(hist.history['accuracy'], label='Training Accuracy')
    ax[i].plot(hist.history['val_accuracy'], label='Validation Accuracy')
    ax[i].legend(loc='lower right')
    print(labels[i]+" vs happy: trained (val. acc. = "+history['val_accuracy'][-1]+")")
    modelname = "from_" + labels[keep[1]]+"_to_"+labels[keep[0]]+".h5"
    model.save(modelname)
    print(modelname,"Saved")
    
    del model
    gc.collect()
plt.show()